# 🔧 Primera Prueba: Métricas de Comparación General

En esta sección se describe la primera batería de pruebas de rendimiento diseñadas para comparar el uso de recursos entre una máquina virtual (**VM**) y un contenedor **Docker**.

El objetivo de estas pruebas es obtener métricas detalladas sobre cómo cada entorno gestiona la carga de trabajo, enfocándose en CPU, memoria, disco y red.

---

## 🛠️ Metodología de Prueba

Se desarrolló un script en Python que ejecuta de forma secuencial las siguientes herramientas de benchmarking:

- **sysbench** → mide el rendimiento de la CPU.
- **stress** → simula carga intensiva sobre el procesador.
- **dd** → evalúa la velocidad de escritura en disco.
- **iperf3** → mide la velocidad de transmisión y recepción de datos en red.

---

## 📊 Recursos Monitoreados

Durante la ejecución de cada prueba, el script monitorizó en tiempo real los siguientes parámetros:

- Porcentaje de uso de **CPU**
- **Memoria RAM** utilizada
- Espacio de **disco** ocupado
- Datos de **red** enviados y recibidos

Esto se logró utilizando la librería **psutil**, que permite obtener estadísticas del sistema con precisión y registrar muestras programadas (una muestra por segundo).

---

## 📁 Almacenamiento de Resultados

Todos los resultados generados por las pruebas fueron guardados en archivos `.txt` dentro de la carpeta de resultados. Cada archivo contiene:

- La salida detallada del test ejecutado.
- Las estadísticas de monitoreo registradas segundo a segundo durante la ejecución.

---

## 🎯 Propósito

Estos datos sirven como base para realizar un análisis comparativo entre ambos entornos (VM vs Docker) y entender mejor las diferencias en eficiencia, manejo de recursos y rendimiento bajo carga.

Los resultados obtenidos serán presentados y analizados en detalle en una sección posterior del informe.


In [1]:
#Imports y configuración inicial
import os
import subprocess
import psutil
import time
from datetime import datetime

# Configuración: ¿es VM o Docker?
vm = False  # Cambia a True si estás ejecutando en una VM

# Crear carpetas
os.makedirs("test", exist_ok=True)
os.makedirs("resultados", exist_ok=True)


In [2]:
#Instalar herramientas necesarias
print("🔧 Instalando herramientas necesarias...")
instalador = ["apt", "install", "-y", "sysbench", "stress", "iperf3"]
if vm:
    instalador.insert(0, "sudo")
subprocess.run(["apt", "update"], check=True) if not vm else subprocess.run(["sudo", "apt", "update"], check=True)
subprocess.run(instalador, check=True)


🔧 Instalando herramientas necesarias...
Hit:1 http://deb.debian.org/debian bookworm InRelease
Hit:2 http://deb.debian.org/debian bookworm-updates InRelease
Hit:3 http://deb.debian.org/debian-security bookworm-security InRelease


Reading package lists...
Building dependency tree...
Reading state information...
All packages are up to date.
Reading package lists...


Building dependency tree...
Reading state information...
sysbench is already the newest version (1.0.20+ds-5).
stress is already the newest version (1.0.7-1).
iperf3 is already the newest version (3.12-1+deb12u1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


CompletedProcess(args=['apt', 'install', '-y', 'sysbench', 'stress', 'iperf3'], returncode=0)

In [3]:
# Definir los tests
tests = {
    "sysbench_cpu": ["sysbench", "cpu", "--cpu-max-prime=20000", "run"],
    "stress_cpu": ["stress", "--cpu", "2", "--timeout", "10"],
    "dd_disk": ["dd", "if=/dev/zero", "of=test/file1.img", "bs=1M", "count=1024", "oflag=dsync"],
    "iperf3_server": ["iperf3", "-s"],
    "iperf3_client": ["iperf3", "-c", "localhost", "-t", "5"]
}


In [4]:
#Función de monitoreo y ejecución
def monitorear_y_ejecutar(comando, duracion_estimada, nombre_archivo):
    uso_recursos = []
    print(f"⏳ Ejecutando y monitoreando: {comando}")

    proceso = subprocess.Popen(comando, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    while proceso.poll() is None:
        stats = {
            "timestamp": datetime.now().strftime('%H:%M:%S'),
            "cpu_percent": psutil.cpu_percent(interval=1),
            "mem_used_mb": psutil.virtual_memory().used / (1024 ** 2),
            "disk_used_gb": psutil.disk_usage('/').used / (1024 ** 3),
            "net_sent_mb": psutil.net_io_counters().bytes_sent / (1024 ** 2),
            "net_recv_mb": psutil.net_io_counters().bytes_recv / (1024 ** 2)
        }
        uso_recursos.append(stats)

    salida, _ = proceso.communicate()

    with open(f"resultados/{nombre_archivo}.txt", "w", encoding="utf-8") as f:
        f.write(f"=== SALIDA DE LA PRUEBA {nombre_archivo} ===\n\n")
        f.write(salida)
        f.write("\n\n=== MONITOREO DE RECURSOS ===\n")
        for r in uso_recursos:
            f.write(f"{r['timestamp']} - CPU: {r['cpu_percent']}% | MEM: {r['mem_used_mb']:.2f} MB | "
                    f"DISK: {r['disk_used_gb']:.2f} GB | NET↑: {r['net_sent_mb']:.2f} MB | NET↓: {r['net_recv_mb']:.2f} MB\n")

    print(f"✅ {nombre_archivo} finalizado y guardado.\n")


In [5]:
#Ejecutar los tests
for test_name, comando in tests.items():
    if test_name == "iperf3_server":
        server_proc = subprocess.Popen(comando, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print("🌐 Servidor iperf3 iniciado...")
        time.sleep(2)
        continue
    elif test_name == "iperf3_client":
        monitorear_y_ejecutar(comando, duracion_estimada=6, nombre_archivo=test_name)
        if 'server_proc' in locals():
            server_proc.terminate()
            print("🛑 Servidor iperf3 detenido.")
    else:
        monitorear_y_ejecutar(comando, duracion_estimada=10, nombre_archivo=test_name)

print("🎉 Todas las pruebas han finalizado correctamente.")


⏳ Ejecutando y monitoreando: ['sysbench', 'cpu', '--cpu-max-prime=20000', 'run']
✅ sysbench_cpu finalizado y guardado.

⏳ Ejecutando y monitoreando: ['stress', '--cpu', '2', '--timeout', '10']
✅ stress_cpu finalizado y guardado.

⏳ Ejecutando y monitoreando: ['dd', 'if=/dev/zero', 'of=test/file1.img', 'bs=1M', 'count=1024', 'oflag=dsync']
✅ dd_disk finalizado y guardado.

🌐 Servidor iperf3 iniciado...
⏳ Ejecutando y monitoreando: ['iperf3', '-c', 'localhost', '-t', '5']
✅ iperf3_client finalizado y guardado.

🛑 Servidor iperf3 detenido.
🎉 Todas las pruebas han finalizado correctamente.


In [7]:
#Graficar los resultados
import matplotlib.pyplot as plt

# Función para cargar datos de un archivo
def cargar_datos_resultado(nombre_archivo):
    timestamps = []
    cpu = []
    mem = []
    ruta = f"resultados/{nombre_archivo}.txt"

    with open(ruta, "r", encoding="utf-8") as f:
        lines = f.readlines()
        monitoreo = [line for line in lines if "CPU:" in line]
        for line in monitoreo:
            parts = line.strip().split(" | ")
            timestamp = parts[0].split(" - ")[0]
            cpu_val = float(parts[0].split("CPU: ")[1].replace("%", ""))
            mem_val = float(parts[1].split("MEM: ")[1].replace(" MB", ""))
            timestamps.append(timestamp)
            cpu.append(cpu_val)
            mem.append(mem_val)
    return timestamps, cpu, mem

# Seleccionar un test para graficar
test_a_graficar = "sysbench_cpu"
timestamps, cpu, mem = cargar_datos_resultado(test_a_graficar)

# Graficar CPU y memoria
plt.figure(figsize=(12, 6))
plt.plot(timestamps, cpu, label='CPU (%)', marker='o')
plt.plot(timestamps, mem, label='Memoria (MB)', marker='x')
plt.xlabel('Tiempo')
plt.ylabel('Uso')
plt.title(f'Resultados del test: {test_a_graficar}')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'matplotlib'